<a href="https://colab.research.google.com/github/not-adev/callAnalyser/blob/main/Callanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q "faster-whisper @ git+https://github.com/guillaumekln/faster-whisper.git" yt-dlp ffmpeg-python
#installing neccasry libraries and modules


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.7 MB/s eta 0:00:00


In [2]:
# library to download the required youtube video mentioned in assigment
import yt_dlp

def download_audio(youtube_url, output_path="audio.wav"): #funtion to downloand
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': output_path,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

# Example usage
download_audio("https://www.youtube.com/watch?v=4ostqJD3Psc") #calling funtion to download

[youtube] Extracting URL: https://www.youtube.com/watch?v=4ostqJD3Psc
[youtube] 4ostqJD3Psc: Downloading webpage
[youtube] 4ostqJD3Psc: Downloading tv simply player API JSON
[youtube] 4ostqJD3Psc: Downloading tv client config
[youtube] 4ostqJD3Psc: Downloading player 0e6689e2-main
[youtube] 4ostqJD3Psc: Downloading tv player API JSON
[info] 4ostqJD3Psc: Downloading 1 format(s): 251
[download] Sleeping 2.00 seconds as required by the site...
[download] Destination: audio.wav
[download] 100% of    1.99MiB in 00:00:00 at 7.49MiB/s   
[ExtractAudio] Destination: audio.wav.wav
Deleting original file audio.wav (pass -k to keep)


In [3]:
from faster_whisper import WhisperModel # importing whisper model for speech to text conversrion
model = WhisperModel("base", device="cpu") #model intitalization


In [4]:
segments,info = model.transcribe("audio.wav.wav", vad_filter=True ,word_timestamps=True ) # converting speech to text with time stamp

In [5]:
transcript = [] # list to store speaker and their text
keywords = ("hi", "hello", "thank")
for seg in segments: #looping over all text or speeches in the video
    speaker = "Speaker A" if any(kw in seg.text.lower() for kw in keywords) else "Speaker B" # identifying caller or reciever
    transcript.append({
        "start": seg.start,
        "end": seg.end,
        "text": seg.text,
        "speaker": speaker
    })


In [8]:
from collections import defaultdict # defalut dictionary to store time of each speaker


In [9]:
talk_time = defaultdict(float) # initialyzing disctionary


In [10]:
for seg in transcript : # loopring over speaker text to find the total time taken by each speaker
  talk_time[seg["speaker"]] += seg["end"] - seg["start"]

In [11]:
total_time = sum(talk_time.values()) # total time of conversation

In [12]:
for i in talk_time : # identifying ratio of each speaker
  print(f'{i} : {talk_time[i]/total_time*100:.2f}%')

Speaker A : 41.04%
Speaker B : 58.96%


In [13]:
Number_of_question = 0 # initializing number of question

In [14]:
for seg in transcript: # looping of transcript to find if their is any question found in conversation
  if "?" in seg["text"] : # if ? found in sentence increase the number of question
    Number_of_question += 1
print(f'number of question asked {Number_of_question}')



number of question asked 8


In [15]:
Longest_monolog = {"speaker" : "null" , "time" : 0 } # dictonary to identify spaeker and longest monolog
for speaker in talk_time: # looping over  talk_time of each speaker
  if talk_time[speaker] > Longest_monolog["time"] : # if speaker talk time is greater then current longest monolog then change speaker and monologe time
    Longest_monolog["time"] = talk_time[speaker]
    Longest_monolog["speaker"] = speaker
print(f'Longest monolog is {Longest_monolog["time"]} by {Longest_monolog["speaker"]} ')



Longest monolog is 54.459999999999994 by Speaker B 


In [16]:
from transformers import pipeline # importing library for sentiment analysis

In [17]:
model_for_sentiment = pipeline("sentiment-analysis") # initializing modle

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [18]:
text = " ".join(seg["text"] for seg in transcript) # convert all text into a single string

In [19]:
sentimenst = model_for_sentiment(text[:512])[0] # extract the firts element from the sentiment
print(f"sentiment of the entire conversation is {sentimenst['label']} with score fo {sentimenst['score']}")


sentiment of the entire conversation is POSITIVE with score fo 0.9986336827278137


In [20]:
if talk_time["Rep"]/total_time > 0.7: # simple insite machanizm
    print("Insight: Let the customer speak more.")
elif Number_of_question < 3:
    print("Insight: Ask more open-ended questions.")
else:
    print("Insight: Good engagement. Consider summarizing key points.")

Insight: Good engagement. Consider summarizing key points.


In [ ]:
print("speaker A is reciver and speaker B is the caller ")